# Deep Learning Neural Network on Animal Shelter data

In [1]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import theano

import matplotlib.pyplot as plt
import os


Using Theano backend.


In [2]:
# Set up my data directories from different machines

mac_data_dir = '/Users/christopherallison/Documents/Coding/Data'
linux_data_dir = '/home/chris/data'
win_data_dir = u'C:\\Users\\Owner\\Documents\\Data'

# Set data directory for example

data_dir = linux_data_dir

In [3]:
# Load our prepared dataset and reference data

df = pd.read_csv(os.path.join(data_dir, "prepared_animals_df.csv"),index_col=0)

In [4]:
df.head()

,OutcomeType,AnimalType,AgeuponOutcome,Color,Intact,Gender,NameLength,BreedKMeans
0,Return_to_owner,1,1,0,0,1,7,0
1,Euthanasia,0,1,1,0,0,5,0
2,Adoption,1,2,2,0,1,6,1
3,Transfer,0,0,2,1,1,3,0
4,Transfer,1,2,3,0,1,3,0


In [5]:
# Drop uneccesary columns
X = df.drop('OutcomeType', axis=1)
X.dtypes

AnimalType        int64
AgeuponOutcome    int64
Color             int64
Intact            int64
Gender            int64
NameLength        int64
BreedKMeans       int64
dtype: object

In [6]:
# We now have a dataframe with 7 features.

X.head()

,AnimalType,AgeuponOutcome,Color,Intact,Gender,NameLength,BreedKMeans
0,1,1,0,0,1,7,0
1,0,1,1,0,0,5,0
2,1,2,2,0,1,6,1
3,0,0,2,1,1,3,0
4,1,2,3,0,1,3,0


In [7]:
X = X.as_matrix()

In [8]:
outcomes = df.OutcomeType.unique()

In [9]:
from sklearn import preprocessing

# This code takes our text labels and creates an encoder that we use
# To transform them into an array

encoder = preprocessing.LabelEncoder()
encoder.fit(outcomes)

encoded_y = encoder.transform(outcomes)
encoded_y

#We can also inverse_transform them back.
list(encoder.inverse_transform([0, 1, 2, 3, 4]))

#We still need to transform the array into a matrix - this is called one hot encoding. It allows us to track the probability of each possible outcome separately.
# convert integers into dummy variables (one hot encoding)

from keras.utils import np_utils

# We'll practice with the dummy for now, but we'll transform the full labels later.
dummy_y = np_utils.to_categorical(encoded_y)
dummy_y

#We'll transform the labels into their array value, though.
df.OutcomeType = encoder.transform(df.OutcomeType)

In [10]:
train_target = np_utils.to_categorical(df['OutcomeType'].values)
train_target

array([[ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.]])

In [17]:
model = Sequential()
model.add(Dense(5, input_dim=7, init='normal', activation="relu"))
model.add(Dense(5, init='normal', activation='sigmoid'))

# Compile model
print("Compiling model...")
model.compile(loss='categorical_crossentropy', optimizer='adam',
             metrics=['accuracy'])

Compiling model...


In [18]:
hist = model.fit(X, train_target, validation_split=0.2)
print("")
print(hist.history)

Train on 21383 samples, validate on 5346 samples
Epoch 1/10
21383/21383 [==============================] - 0s - loss: 1.2948 - acc: 0.3948 - val_loss: 1.2288 - val_acc: 0.3992
Epoch 2/10
21383/21383 [==============================] - 0s - loss: 1.1645 - acc: 0.4662 - val_loss: 1.0900 - val_acc: 0.5617
Epoch 3/10
21383/21383 [==============================] - 0s - loss: 1.0264 - acc: 0.5865 - val_loss: 1.0042 - val_acc: 0.5864
Epoch 4/10
21383/21383 [==============================] - 0s - loss: 0.9960 - acc: 0.5890 - val_loss: 0.9894 - val_acc: 0.5847
Epoch 5/10
21383/21383 [==============================] - 0s - loss: 0.9870 - acc: 0.5885 - val_loss: 0.9848 - val_acc: 0.5922
Epoch 6/10
21383/21383 [==============================] - 0s - loss: 0.9818 - acc: 0.5912 - val_loss: 0.9760 - val_acc: 0.5860
Epoch 7/10
21383/21383 [==============================] - 0s - loss: 0.9772 - acc: 0.5915 - val_loss: 0.9728 - val_acc: 0.5898
Epoch 8/10
21383/21383 [==============================] - 0s -

In [13]:
model.evaluate(X, train_target)

26720/26729 [============================>.] - ETA: 0s

[1.0065278678596157, 0.58397246436900774]

In [14]:
model.predict_proba(X[0:2])

2/2 [==============================] - 0s


array([[  7.15802908e-01,   3.10082076e-04,   1.21944146e-02,
          2.80688465e-01,   1.35272399e-01],
       [  4.00894046e-01,   2.09603552e-03,   2.74598897e-02,
          1.04723938e-01,   3.01309079e-01]], dtype=float32)